THIS FUNCTION UPDATE THE DATETIME FIRST VALUES OF EACH SPEECH WITH THE CORRECTED VALUES EXTRACTED FROM THE HTML DATAFRAME 

In [72]:
import warnings
warnings.simplefilter("ignore")


In [73]:
import pandas as pd
from helpermodules import memory_handling as mh
import numpy as np
from datetime import timedelta, datetime
import time

In [74]:
# downloading the pickle file 
file = "fedspeeches_preprocessed.pkl"
helper = mh.PickleHelper.pickle_load(file)
df_speech = helper.obj

In [75]:
speaker_list_riccardo = [ 'Vice Chair Janet L. Yellen', 'Governor Sarah Bloom Raskin', 'Governor Jeremy C. Stein', 'Governor Jerome H. Powell', 'Chairman  Ben S. Bernanke', 'Governor Lael Brainard', 'Chair Janet L. Yellen', 'Vice Chairman Stanley Fischer', 'Vice Chairman for Supervision Randal K. Quarles', 'Chairman Jerome H. Powell', 'Vice Chairman Richard H. Clarida', 'Chair Jerome H. Powell', 'Vice Chair Richard H. Clarida', 'Vice Chair for Supervision Randal K. Quarles']
speaker_list_fabio = ['Governor Michelle W. Bowman', 'Vice Chair for Supervision and Chair of the Financial Stability Board Randal K. Quarles', 'Vice Chairman for Supervision and Chair of the Financial Stability Board Randal K. Quarles', 'Governor Christopher J. Waller', 'Governor Randal K. Quarles', 'Vice Chair for Supervision Michael S. Barr', 'Governor Lisa D. Cook', 'Vice Chair Lael Brainard', 'Governor Philip N. Jefferson', 'Chair Pro Tempore Jerome H. Powell', 'Vice Chair Philip N. Jefferson', 'Governor Adriana D. Kugler']
df_speech = df_speech[df_speech['speaker'].isin(speaker_list_fabio)]

In [76]:
trial = pd.DataFrame({
    'speaker': ['Governor Michelle W. Bowman' , 'Governor Christopher J. Waller'],
    'date': ['2024-10-23' , '2024-10-14'],
    'opening_time': ['2024-10-23 14:00:00+02:00' , '2024-10-23 15:03:00+02:00' ],
    'title': ['Opening Remarks' , 'Thoughts on the Economy and Policy Rules at the Federal Open Market Committee']
})

trial['date'] = pd.to_datetime(trial['date'])
trial['opening_time'] = pd.to_datetime(trial['opening_time'])

In [77]:
trial

,speaker,date,opening_time,title
0,Governor Michelle W. Bowman,2024-10-23,2024-10-23 14:00:00+02:00,Opening Remarks
1,Governor Christopher J. Waller,2024-10-14,2024-10-23 15:03:00+02:00,Thoughts on the Economy and Policy Rules at th...


In [78]:
#eliminating timezone info on the original dataframe as it's a big mess otherwise 

df_speech['timestamp'] = df_speech['timestamp'].dt.tz_localize(None)
df_speech['date'] = pd.to_datetime(df_speech['date']).dt.date

In [79]:
def change_time(df_speech, dfnew):
    """
    Adjusts the timestamps in the `df_speech` DataFrame based on the opening time
    from `dfnew` for each speech. Specifically, it updates the timestamp of the
    first event for each unique combination of speaker, date, and title to the
    opening time specified in `dfnew`, and shifts subsequent timestamps by one minute.

    Parameters:
    df_speech (pd.DataFrame): DataFrame containing speech details with columns like
                               'speaker', 'date', 'title', and 'timestamp'.
    dfnew (pd.DataFrame): DataFrame containing speech details with columns 'speaker',
                          'date', 'title', and 'opening_time' (the time when the speech begins).

    Returns:
    pd.DataFrame: A modified version of `df_speech` where the timestamps are updated
                  based on the opening time for matching speeches, and rows with no matches
                  are removed. The rows where the "timestamp" was updated will remain.

    Side Effects:
    - Adds a "check" column to track which rows have been updated.
    - Drops rows that didn't have a matching speech in `dfnew`.
    - Prints the drop ratio, which is the ratio of remaining unique texts after the update.
    """  


    #ensuring the time columns share the same datetime format 
    df_speech.loc[:, 'date'] = pd.to_datetime(df_speech['date'])
    dfnew['date'] = pd.to_datetime(dfnew['date'])

    # Add a "check" column to track updated rows
    df_speech.loc[:, 'check'] = 0

    # Count the initial number of unique texts
    r2 = df_speech['text'].nunique()

    for i in range(len(dfnew)):
        # Extract details for the current speech
        speaker = dfnew.iloc[i]['speaker']
        date = dfnew.iloc[i]['date']
        title = dfnew.iloc[i]['title']
        newtime = dfnew.iloc[i]['opening_time']

        # Find rows in df_speech matching speaker, date, and title
        mask = (df_speech['speaker'] == speaker) & (df_speech['date'] == date) & (df_speech['title'] == title)

        # Update the minimum timestamp and set the "check" flag
        if mask.any():
            min_idx = df_speech.loc[mask, 'timestamp'].idxmin()
            df_speech.at[min_idx, 'timestamp'] = newtime
            df_speech.loc[mask, 'timestamp'] = (
                newtime + pd.to_timedelta(range(len(df_speech[mask])), unit='min')
            )
            df_speech.loc[mask, 'check'] = 1

    # Drop rows that were not updated (check == 0)
    df_speech = df_speech[df_speech['check'] == 1].drop(columns=['check'])

    # Count the remaining unique texts and calculate the drop ratio
    r1 = df_speech['text'].nunique()
    print('The drop ratio is', (r1 / r2)*100,'%')

    return df_speech

In [80]:
df1 = change_time(df_speech , trial)
df1

The drop ratio is 0.7490636704119851 %


,date,speaker,title,link,text,timestamp,text_by_minute
22470,2024-10-23 00:00:00,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 14:00:00+02:00,Good morning.1 It is a pleasure to join you fo...
22471,2024-10-23 00:00:00,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 14:01:00+02:00,benefits and risks by bringing together expert...
22472,2024-10-23 00:00:00,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 14:02:00+02:00,"new technology. Second, we must be open in con..."
22473,2024-10-23 00:00:00,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 14:03:00+02:00,the future.But we must recognize that the succ...
22474,2024-10-23 00:00:00,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 14:04:00+02:00,"with President Harker for the past six years, ..."
22475,2024-10-23 00:00:00,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 14:05:00+02:00,the application of both hard and soft data.Thi...
22476,2024-10-23 00:00:00,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 14:06:00+02:00,Federal Open Market Committee.
22494,2024-10-14 00:00:00,Governor Christopher J. Waller,Thoughts on the Economy and Policy Rules at th...,/newsevents/speech/waller20241014a.htm,"Thank you, Athanasios, and thank you for the o...",2024-10-23 15:03:00+02:00,"Thank you, Athanasios, and thank you for the o..."
22495,2024-10-14 00:00:00,Governor Christopher J. Waller,Thoughts on the Economy and Policy Rules at th...,/newsevents/speech/waller20241014a.htm,"Thank you, Athanasios, and thank you for the o...",2024-10-23 15:04:00+02:00,"the most recent FOMC meeting, data we have rec..."
22496,2024-10-14 00:00:00,Governor Christopher J. Waller,Thoughts on the Economy and Policy Rules at th...,/newsevents/speech/waller20241014a.htm,"Thank you, Athanasios, and thank you for the o...",2024-10-23 15:05:00+02:00,"overstating the strength of the economy, since..."
